# Image Segmenter

In [1]:
!pip install -q mediapipe

DEPRECATION: wfuzz 3.1.0 has a non-standard dependency specifier pyparsing>=2.4*; python_version >= "3.5". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of wfuzz or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Download the image segmenter model

The next thing you will need to do is download the image segmentation model that will be used for this demo. In this case we will use the DeepLab v3 model.

In [1]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/image_segmenter/deeplab_v3/float32/1/deeplab_v3.tflite"
file_path = "deeplabv3.tflite"

response = requests.get(url)
with open(file_path, "wb") as file:
    file.write(response.content)


## Add a test image

After downloading the model, it's time to grab an image that you can use for testing! It's worth noting that while this is working with a single image, you can download a collection of images to store in the `IMAGE_FILENAMES` array.

In [57]:
import urllib

IMAGE_FILENAMES = ['bilal2.jpg'] #INSERT JPG FILE PATH HERE. ONE IMAGE PATH


## Preview the downloaded image

With the test image downloaded, go ahead and display it.

In [51]:
import cv2
import math

# Height and width that will be used by the model
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

# Performs resizing and showing the image
def resize_and_show(image):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h / (w / DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w / (h / DESIRED_HEIGHT)), DESIRED_HEIGHT))
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Preview the image(s)
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}
for name, image in images.items():
    print(name)
    resize_and_show(image)


bilal2.jpg


## Running inference and visualizing the results

In [52]:
import numpy as np
import mediapipe as mp

from mediapipe.tasks import python
from mediapipe.tasks.python import vision


BG_COLOR = (0, 0, 0) # black
MASK_COLOR = (255, 255, 255) # white


# Create the options that will be used for ImageSegmenter
base_options = python.BaseOptions(model_asset_path='deeplabv3.tflite')
options = vision.ImageSegmenterOptions(base_options=base_options,
                                       output_category_mask=True)

print(IMAGE_FILENAMES)
# Create the image segmenter
with vision.ImageSegmenter.create_from_options(options) as segmenter:

  # Loop through demo image(s)
  for image_file_name in IMAGE_FILENAMES:

    # Create the MediaPipe image file that will be segmented
    image = mp.Image.create_from_file(image_file_name)

    # Retrieve the masks for the segmented image
    segmentation_result = segmenter.segment(image)
    category_mask = segmentation_result.category_mask

    # Generate solid color images for showing the output segmentation mask.
    image_data = image.numpy_view()
    fg_image = np.zeros(image_data.shape, dtype=np.uint8)
    fg_image[:] = MASK_COLOR
    bg_image = np.zeros(image_data.shape, dtype=np.uint8)
    bg_image[:] = BG_COLOR

    condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
    output_image = np.where(condition, fg_image, bg_image)

    output_file_name = f"{image_file_name.split('.')[0]}_silhouette.jpg"
    cv2.imwrite(output_file_name, output_image)

    print(f"Saved silhouette image as: {output_file_name}")

    print(f'Segmentation mask of {name}:')
    resize_and_show(output_image)

['bilal2.jpg']
Saved silhouette image as: bilal2_silhouette.jpg
Segmentation mask of bilal2.jpg:


In [29]:
from tensorflow.keras.models import load_model

model = load_model("final_model.h5")


In [53]:
def preprocess_image(image):
    # Resize image to (64, 64)
    resized_image = cv2.resize(image, (128, 128))
    
    # Convert to RGBA format
    rgba_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGBA)

    # Convert to float32 and normalize
    normalized_image = rgba_image.astype(np.float32) / 255.0

    # Add batch dimension
    preprocessed_image = np.expand_dims(normalized_image, axis=0)

    return preprocessed_image

def postprocess_image(image):
    return image*256

In [54]:
final_image = preprocess_image(output_image)


In [55]:
data = postprocess_image(model.predict(final_image))

1/1 [==============================] - 0s 35ms/step


In [56]:
from tabulate import tabulate


# Headers in the same order as the data
headers = ["ankle", "arm-length", "bicep", "calf", "chest", "forearm", "height",
           "hip", "leg-length", "shoulder-breadth", "shoulder-to-crotch", "thigh", "waist", "wrist"]


# Present the data in tabular format
table = tabulate(data, headers=headers, tablefmt="grid")

print("The predicted body measurements in cm are:")
print(table)


The predicted body measurements in cm are:
+---------+--------------+---------+--------+---------+-----------+----------+---------+--------------+--------------------+----------------------+---------+---------+---------+
|   ankle |   arm-length |   bicep |   calf |   chest |   forearm |   height |     hip |   leg-length |   shoulder-breadth |   shoulder-to-crotch |   thigh |   waist |   wrist |
+=========+==============+=========+========+=========+===========+==========+=========+==============+====================+======================+=========+=========+=========+
| 22.0463 |      35.4276 | 26.4882 | 31.174 | 86.3326 |   20.5838 |  144.919 | 91.5133 |      63.7362 |            31.2414 |              59.4454 | 49.3582 | 78.7543 |  11.074 |
+---------+--------------+---------+--------+---------+-----------+----------+---------+--------------+--------------------+----------------------+---------+---------+---------+
